In [1]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

In [2]:
# CHANGE ME!
TASK_ID = 'tom_hanks'
AUTHOR = 'g'


In [3]:
task = Task(
    id=TASK_ID,
    name="Directors of movies Tom Hanks starred in",
    description="Find directors of movies that Tom Hanks starred in",
    plan=[
        {
        "id": "director",
        "description": "Find director",
    }, {
        "id": "actor",
        "description": "Tom Hanks starred in",
    } 
    ],
    sample_input={
        "actors":[
            {"movie":"AM","actor":"Tom Hanks"},
            {"movie":"AM","actor":"Not Tom Hanks"},
            {"movie":"AM","actor":"Also not Tom Hanks"},
            {"movie":"BM","actor":"Other actor"},
            {"movie":"CM","actor":"Other actor"},
        ],
        "directors":[
            {"director":"A","movie":"AM"},
            {"director":"A","movie":"BM"},
            {"director":"C","movie":"CM"},
        ],
    },
    sample_output=["A"],
    category='Basic'
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [4]:
imp = replace(prototype,
    language='python-imperative',
    source='''
def tom_hanks(actors,directors):
  r_directors = set()
  for a in actors:
    if 'Tom Hanks' == a['actor']:
      for d in directors:
        if a['movie'] == d['movie']:
          r_directors.add(d['director'])
    return r_directors
''').load_plan()
imp.execute(task)
imp.save()

In [5]:
func = replace(prototype,
    language='python-functional',
    source='''
def tom_hanks(actors,directors):
  movies = [a['movie'] for a in actors \
    if a['actor'] == 'Tom Hanks']
  directors = [d['director'] for d in directors \
    if d['movie'] in movies]
  return directors
''').load_plan()
func.execute(task)
func.save()

In [6]:
pandas = replace(prototype,
    language='python-pandas',
    source='''
def tom_hanks(actors,directors):
  movies = list(actors.loc[
    actors['actor'] == 'Tom Hanks', 'movie'])
  return directors[directors['movie'].
    isin(movies)]['director']
''').load_plan()
pandas.execute(task)
pandas.save()

In [7]:
sql = replace(prototype,
    language='sql',
    source='''
SELECT directors.director
FROM 
  directors 
  INNER JOIN actors
  ON directors.movie = 
  actors.movie
WHERE actors.actor="Tom Hanks"
''').load_plan()
sql.execute(task)
sql.save()

In [8]:
datalog = replace(prototype,
    language='datalog',
    source='''
tom_hanks(Director):-
  actors("Tom Hanks",movie),
  directors(Director,movie).
''').load_plan()
datalog.execute(task)
datalog.save()

In [13]:
program = imp
program.widget(task)

Output()

CodeWidget(program='{"task": "tom_hanks", "language": "python-imperative", "plan": {}, "source": "\\ndef tom_h…